#### Lab1：Regression

In [21]:
import pandas as pd
# load train_set.csv
train_set = pd.read_csv('train_set.csv')
# load test_set.csv
test_set = pd.read_csv('test_set.csv')

# check the shape of train_set & test_set
print("Shape of train_set:", train_set.shape, "; Shape of test_set:", test_set.shape)
# describe the train set
train_set.describe()

Shape of train_set: (15726, 13) ; Shape of test_set: (3949, 13)


,median_income,house_age,avg_rooms,avg_bedrooms,block_population,avg_occupancy,latitude,longitude,house_value,diag_coord,bed_per_rooms,rooms_per_occup,pop_per_occup
count,15726.000000,15726.000000,15726.000000,15726.000000,15726.000000,15726.000000,15726.000000,15726.000000,15726.000000,15726.000000,15726.000000,15726.000000,15726.000000
mean,3.678555,28.379944,5.370023,1.098158,1435.302620,3.123054,35.657075,-119.566030,1.923611,-83.908955,0.214882,1.943798,498.807135
std,1.570925,12.497755,2.398827,0.480437,1127.179163,11.864913,2.152346,2.003633,0.977266,0.822970,0.056292,1.161179,377.719409
min,0.499900,1.000000,1.130435,0.333333,3.000000,0.692308,32.540000,-124.350000,0.149990,-85.890000,0.100000,0.002547,2.000000
25%,2.535050,18.000000,4.416667,1.005640,793.000000,2.446153,33.930000,-121.760000,1.167000,-84.410000,0.177622,1.497171,281.000000
50%,3.454400,28.000000,5.187500,1.048420,1177.000000,2.838178,34.280000,-118.520000,1.733500,-84.160000,0.204512,1.912297,409.500000
75%,4.579400,37.000000,5.972678,1.098845,1740.750000,3.304575,37.730000,-118.000000,2.475000,-83.460000,0.241086,2.249429,603.000000
max,15.000100,52.000000,132.533333,34.066667,35682.000000,1243.333333,41.950000,-114.310000,5.000000,-78.290000,1.000000,55.222222,5358.000000


In [22]:
# split input and output of both train_set and test_set
train_x = train_set.drop(['house_value'], axis=1)
train_y = train_set['house_value']
test_x = test_set.drop(['house_value'], axis=1)
test_y = test_set['house_value']
# transform the sets into numpy array
train_x = train_x.values
train_y = train_y.values
test_x = test_x.values
test_y = test_y.values

In [23]:
# import sklearn's linear, lasso, ridge, SVM, k-neighbors, descision tree, GBDT regression
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from myRegression import myRegression

# import sklearn's mean squared error and r2 score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

import time

def train_model(model, train_x, train_y):
    # start timing
    start_time = time.time()
    # train the model
    model.fit(train_x, train_y)
    # end timing
    end_time = time.time()
    training_time = end_time - start_time
    # calculate the MSE, r2 & adjested r2 score of training set
    pred_y = model.predict(train_x)
    mse = mean_squared_error(train_y, pred_y)
    r2 = r2_score(train_y, pred_y)
    adj_r2 = 1 - (1 - r2) * (len(train_y) - 1) / (len(train_y) - train_x.shape[1] - 1)
    return training_time, mse, r2, adj_r2
    

def test_model(model, test_x, test_y):
    # predict the output of test set
    pred_y = model.predict(test_x)
    # calculate the mean squared error
    mse = mean_squared_error(test_y, pred_y)
    # calculate the r2 score
    r2 = r2_score(test_y, pred_y)
    # calcultae the adjusted r2 score
    adj_r2 = 1 - (1 - r2) * (len(test_y) - 1) / (len(test_y) - test_x.shape[1] - 1)
    return mse, r2, adj_r2


# create a function to train and test the models
def train_test_model(model, train_x, train_y, test_x, test_y):
    # train the model
    train_result = train_model(model, train_x, train_y)
    # test the model
    test_result = test_model(model, test_x, test_y)
    # concat train & test result
    result = train_result + test_result
    return result



In [24]:
# create a dict with all the models
models = {
    'MyRegression': myRegression(),
    'LinearRegression': LinearRegression(),
    'Lasso': Lasso(),
    'Ridge': Ridge(),
    'SVR': SVR(),
    'KNeighborsRegressor': KNeighborsRegressor(),
    'DecisionTreeRegressor': DecisionTreeRegressor(),
    'GradientBoostingRegressor': GradientBoostingRegressor()
}

# create a pandas dataframe to log the results
result_df = pd.DataFrame(columns=['model_name', 'training_time', 'train_mse', 'train_r2', 'train_adj_r2', 'test_mse', 'test_r2', 'test_adj_r2'])

# train and test all the models
for name, model in models.items():
    print(name)
    result = train_test_model(model, train_x, train_y, test_x, test_y)
    # append the name and result to result_df
    result_df = result_df.append({'model_name': name, 'training_time': result[0], 'train_mse': result[1], 'train_r2': result[2], 'train_adj_r2': result[3], 'test_mse': result[4], 'test_r2': result[5], 'test_adj_r2': result[6]}, ignore_index=True)

MyRegression


d:\GitHubMyProject\ML-Lab\lab1\myRegression.py:90: RuntimeWarning: overflow encountered in power
  mse = np.sum(np.power((pred_y - y), 2)) / y.shape[0]
d:\softwares\Python\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
d:\GitHubMyProject\ML-Lab\lab1\myRegression.py:62: RuntimeWarning: overflow encountered in multiply
  update = [np.reshape(error * X[:, j], (-1, 1)) for j in range(len_theta)]
d:\softwares\Python\lib\site-packages\numpy\core\_methods.py:179: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
d:\GitHubMyProject\ML-Lab\lab1\myRegression.py:69: RuntimeWarning: invalid value encountered in subtract
  self.theta = self.theta * (1-alpha*(lamda/num_data)) - alpha*update


KeyboardInterrupt: 

In [ ]:
from IPython.display import display
# display the result_df
display(result_df)
result_df.to_csv('result.csv', index=False)

,model_name,training_time,train_mse,train_r2,train_adj_r2,test_mse,test_r2,test_adj_r2
0,LinearRegression,0.003000,0.359732,0.623313,0.623025,0.349395,0.633585,0.632468
1,Lasso,0.037510,0.913391,0.043558,0.042827,0.912354,0.043204,0.040287
2,Ridge,0.000998,0.359818,0.623223,0.622935,0.348935,0.634068,0.632952
3,SVR,4.638352,0.879537,0.079008,0.078305,0.884729,0.072174,0.069345
4,KNeighborsRegressor,0.014977,0.634212,0.335895,0.335388,0.972041,-0.019391,-0.022499
5,DecisionTreeRegressor,0.146119,0.000000,1.000000,1.000000,0.364770,0.617461,0.616295
6,GradientBoostingRegressor,3.208935,0.198160,0.792500,0.792342,0.210289,0.779467,0.778795
